In [1]:
%matplotlib inline
from collections import defaultdict, Counter
import glob
import os
import json

from IPython.core.display import HTML, Image
from matplotlib_venn import venn3
import pandas as pd
import pybedtools
import pysam
import gspread
from gscripts.general import parsers
from gscripts.general import dataviz
%load_ext autoreload
import numpy as np
%autoreload 2
reload(parsers)
reload(pybedtools)
import seaborn as sns
import matplotlib
from oauth2client.client import SignedJwtAssertionCredentials
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter

img_dir = "/home/gpratt/Dropbox/encode_integration/qc_work/"

/home/gpratt/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
final_qc_frame = pd.read_csv("/home/gpratt/Dropbox/EricGabe_ENCODE/encode_master_qc.csv", index_col=[0,1,2,3,4,5])
encode_qc = pd.read_csv("/home/gpratt/Dropbox/EricGabe_ENCODE/unannoated_encode_master_qc.csv", index_col=0)

In [3]:
filtered_final_qc_frame = final_qc_frame[[ "Input Reads", "Reads Written", "repetitive_count", "Reads Passing Quality Filter",
                                          "Uniquely Mapped Reads", "Uniquely mapped reads %", 'Number of reads mapped to too many loci',
                                          '% of reads unmapped: too short', '% of reads mapped to too many loci', "Usable Reads",
                                          "Fraction Collapsed", "Fraction Usable", "Num Peaks",]]

# CVB Data (From layla)

In [4]:
cvb_frame = unmerged_clip_manifest[["CVB" in item for item in unmerged_clip_manifest.index]]
cvb_frame['percent_repetitive'] = cvb_frame['repetitive_count'] / cvb_frame['Reads Written']
cvb_frame.to_csv("CBV_stats_all.csv")
filtered_cbv_frame = cvb_frame[[ "Input Reads",
                                #'Trimmed bases',
                            "Reads Written",
                            "repetitive_count",
                            'percent_repetitive',
                            "Uniquely Mapped Reads",
                            "Uniquely mapped reads %",
                            "Usable Reads",
                            "Fraction Collapsed",
                            "Fraction Usable",
                            #"spot",
                            "Num Peaks",
                            #"Passed QC"
                            ]]

HTML(filtered_cbv_frame.to_html(formatters={"Input Reads" : parsers.commas,
                                     "Reads Written" : parsers.commas,
                                     "repetitive_count": parsers.commas,
                                     "Reads after Quality Filtering" : parsers.commas,
                                     "Uniquely Mapped Reads" : parsers.commas,
                                     "Usable Reads" : parsers.commas,
                                     "Num Peaks": parsers.commas
                                     } ))

NameError: name 'unmerged_clip_manifest' is not defined

# QC for Kris' data only

Good:
NUMA1 
RANGAP1
ZNF217 
ZNF184 
RNF219 
NUMA1 

Borderline:
RNF219 -- low mapping %, and doesn't hit 1m reads
AIFM1  -- low mapping % and barely hits 1m reads
DHX30 
BAG2 -- Might bind repetitive elements?

Bad:
VIM -- low mapping % doesn't hit 1m reads
AIFM1 -- low mapping % doesn't hit 1m reads
RANGAP1 -- KB1 odd result


In [5]:
kb_filtered_final_qc_frame = filtered_final_qc_frame[['KB' in item for item in filtered_final_qc_frame.index.get_level_values("encode_id")]]
kb_filtered_final_qc_frame.index = kb_filtered_final_qc_frame.index.droplevel(["cell_type"])
kb_filtered_final_qc_frame.index = kb_filtered_final_qc_frame.index.droplevel(["rep"])
kb_filtered_final_qc_frame = kb_filtered_final_qc_frame.drop("repetitive_count", axis=1)
kb_filtered_final_qc_frame['repetitive_count'] = (kb_filtered_final_qc_frame['Reads Written'] - kb_filtered_final_qc_frame['Reads Passing Quality Filter']).astype(int)

HTML(kb_filtered_final_qc_frame.sort_index(level="rbp", ascending=False).to_html(formatters={"Input Reads" : parsers.commas,
                                     "Reads Written" : parsers.commas,
                                     "repetitive_count": parsers.commas,
                                     "Reads after Quality Filtering" : parsers.commas,
                                     "Reads Passing Quality Filter" : parsers.commas,
                                     "Uniquely Mapped Reads" : parsers.commas,
                                     "Usable Reads" : parsers.commas,
                                     "Num Peaks": parsers.commas
                                     } ))

In [6]:
kb_filtered_final_qc_frame.sort_index(level="rbp", ascending=False).to_csv("/home/gpratt/Dropbox/kris_qc_table.csv")

# For Natasha

In [7]:
kb_filtered_final_qc_frame = filtered_final_qc_frame[['NM' in item for item in filtered_final_qc_frame.index.get_level_values("encode_id")]]
kb_filtered_final_qc_frame.index = kb_filtered_final_qc_frame.index.droplevel(["cell_type"])
kb_filtered_final_qc_frame.index = kb_filtered_final_qc_frame.index.droplevel(["rep"])
kb_filtered_final_qc_frame = kb_filtered_final_qc_frame.drop("repetitive_count", axis=1)
kb_filtered_final_qc_frame['repetitive_count'] = (kb_filtered_final_qc_frame['Reads Written'] - kb_filtered_final_qc_frame['Reads Passing Quality Filter']).astype(int)

HTML(kb_filtered_final_qc_frame.sort_index(level="rbp", ascending=False).to_html(formatters={"Input Reads" : parsers.commas,
                                     "Reads Written" : parsers.commas,
                                     "repetitive_count": parsers.commas,
                                     "Reads after Quality Filtering" : parsers.commas,
                                     "Reads Passing Quality Filter" : parsers.commas,
                                     "Uniquely Mapped Reads" : parsers.commas,
                                     "Usable Reads" : parsers.commas,
                                     "Num Peaks": parsers.commas
                                     } ))

In [8]:
kb_filtered_final_qc_frame.sort_index(level="rbp", ascending=False).to_csv("/home/gpratt/Dropbox/Natasha_qc_table.csv")

In [9]:
ron_df = filtered_final_qc_frame.ix['RB01']

HTML(ron_df.to_html(formatters={"Input Reads" : parsers.commas,
                                     "Reads Written" : parsers.commas,
                                     "repetitive_count": parsers.commas,
                                     "Reads after Quality Filtering" : parsers.commas,
                                     "Reads Passing Quality Filter" : parsers.commas,
                                     "Uniquely Mapped Reads" : parsers.commas,
                                     "Usable Reads" : parsers.commas,
                                     "Num Peaks": parsers.commas
                                     } ))

KeyError: 'RB01'

In [ ]:
ron_df.to_csv("/home/gpratt/Dropbox/")

# For Singapore

In [12]:
HTML(filtered_final_qc_frame.to_html())

In [4]:
kb_filtered_final_qc_frame = filtered_final_qc_frame[['CHH' in item for item in filtered_final_qc_frame.index.get_level_values("encode_id")]]
kb_filtered_final_qc_frame.index = kb_filtered_final_qc_frame.index.droplevel(["cell_type"])
kb_filtered_final_qc_frame.index = kb_filtered_final_qc_frame.index.droplevel(["rep"])
kb_filtered_final_qc_frame = kb_filtered_final_qc_frame.drop("repetitive_count", axis=1)
kb_filtered_final_qc_frame['repetitive_count'] = (kb_filtered_final_qc_frame['Reads Written'] - kb_filtered_final_qc_frame['Reads Passing Quality Filter']).astype(int)

HTML(kb_filtered_final_qc_frame.sort_index(level="rbp", ascending=False).to_html(formatters={"Input Reads" : parsers.commas,
                                     "Reads Written" : parsers.commas,
                                     "repetitive_count": parsers.commas,
                                     "Reads after Quality Filtering" : parsers.commas,
                                     "Reads Passing Quality Filter" : parsers.commas,
                                     "Uniquely Mapped Reads" : parsers.commas,
                                     "Usable Reads" : parsers.commas,
                                     "Num Peaks": parsers.commas
                                     } ))

In [5]:
kb_filtered_final_qc_frame.to_csv("/home/gpratt/Dropbox/singapore_clip_master_qc.csv")

In [ ]:
FOX2_AFFSC,FOX2_SS4,FOX2_SS4_MN,FOX2_SS4in3B,FOX2_SS4in3B_AS46,FOX2_SSIII,FOX2_TGIRT,TDP_AFFSC,TDP_SS4,TDP_SS4_MN,TDP_SS4in3B,TDP_SS4in3B_AS48,TDP_SSIII,TDP_TGIRT